# 🤖 Taller Sesión 4: Chatbot con LangChain y Gestión de Contexto

**Curso:** Diseño e Implementación de Chatbots  
**Docente:** Angelo Castillo Meca  
**Sesión:** 4 - Chatbot basado en LLMs

---

## 📋 Objetivo del Taller

Implementar un **asistente conversacional para e-commerce** usando LangChain que pueda:
- Responder preguntas sobre productos
- Procesar consultas de seguimiento
- Generar recomendaciones personalizadas
- Mantener contexto conversacional

## 🎯 Competencias a Desarrollar

1. Usar LangChain para orquestar interacciones con LLMs
2. Implementar gestión de memoria conversacional
3. Diseñar prompts efectivos para chatbots
4. Crear cadenas de procesamiento (Chains)

---

## 📦 Parte 1: Instalación de Dependencias

Instalamos LangChain y las bibliotecas necesarias.

In [ ]:
# Instalar dependencias
!pip install -r requirements.txt

print("✅ Dependencias instaladas correctamente")

## 🔑 Parte 2: Configuración de API Key

**Importante:** Necesitas una API key de OpenAI. Puedes obtenerla en: https://platform.openai.com/api-keys

**Alternativa Gratuita:** Si no tienes API key, puedes usar modelos locales con HuggingFace (ver sección alternativa al final).

In [2]:
import os
import getpass

# Configurar API key de Alibaba DashScope
if "DASHSCOPE_API_KEY" not in os.environ:
    os.environ["DASHSCOPE_API_KEY"] = getpass.getpass("Ingresa tu DashScope API Key: ")
# sk-0f8e7079049345b7bed6fc8d494b995e
# LangChain usa OPENAI_API_KEY, así que la asignamos también
os.environ["OPENAI_API_KEY"] = os.environ["DASHSCOPE_API_KEY"]
    
print("✅ API Key de Alibaba DashScope configurada")

✅ API Key de Alibaba DashScope configurada


## 🔧 Parte 3: Importar Bibliotecas

In [3]:
from langchain_openai import ChatOpenAI
from langchain_classic.chains import ConversationChain
from langchain_classic.memory import (
    ConversationBufferMemory,
    ConversationBufferWindowMemory,
    ConversationSummaryMemory,
    ConversationSummaryBufferMemory,
    ConversationEntityMemory
)
from langchain_classic.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder
from langchain_classic.schema import HumanMessage, AIMessage, SystemMessage
import warnings
warnings.filterwarnings('ignore')

print("✅ Bibliotecas importadas correctamente")

✅ Bibliotecas importadas correctamente


## 🧠 Parte 4: Inicializar el Modelo LLM

Configuramos el modelo de OpenAI con parámetros apropiados para un chatbot.

In [4]:
# Inicializar modelo LLM usando Alibaba DashScope
llm = ChatOpenAI(
    model="qwen-turbo",  # Modelos disponibles: qwen-turbo, qwen-plus, qwen-max, qwen-max-longcontext
    temperature=0.7,     # Control de creatividad (0.0 = conservador, 1.0 = creativo)
    max_tokens=500,      # Máximo de tokens en la respuesta
    base_url="https://dashscope-intl.aliyuncs.com/compatible-mode/v1"  # URL de Alibaba DashScope
)

print("✅ Modelo LLM inicializado con Alibaba DashScope")
print(f"   Modelo: qwen-turbo")
print(f"   Base URL: https://dashscope-intl.aliyuncs.com/compatible-mode/v1")
print(f"   Temperature: 0.7")
print(f"   Max tokens: 500")
print("\n💡 Modelos disponibles de Alibaba:")
print("   - qwen-turbo (rápido y económico)")
print("   - qwen-plus (equilibrado)")
print("   - qwen-max (más potente)")
print("   - qwen-max-longcontext (contexto largo)")

✅ Modelo LLM inicializado con Alibaba DashScope
   Modelo: qwen-turbo
   Base URL: https://dashscope-intl.aliyuncs.com/compatible-mode/v1
   Temperature: 0.7
   Max tokens: 500

💡 Modelos disponibles de Alibaba:
   - qwen-turbo (rápido y económico)
   - qwen-plus (equilibrado)
   - qwen-max (más potente)
   - qwen-max-longcontext (contexto largo)


## 💭 Parte 5: Configurar Memoria Conversacional

La memoria permite que el chatbot recuerde interacciones previas.

In [5]:
# Crear memoria conversacional (buffer completo)
memory = ConversationBufferMemory(
    return_messages=True,  # Retornar como lista de mensajes
    memory_key="chat_history"  # Nombre de la variable en el prompt
)

print("✅ Memoria conversacional configurada")
print("   Tipo: ConversationBufferMemory (almacena todo el historial)")

✅ Memoria conversacional configurada
   Tipo: ConversationBufferMemory (almacena todo el historial)


## 📝 Parte 6: Diseñar Prompt Template

El prompt define el rol y comportamiento del chatbot.

In [4]:
# Template del prompt para el asistente de e-commerce
template = """Eres un asistente virtual amigable y experto para una tienda de tecnología online llamada "TechStore". 

Tu objetivo es ayudar a los clientes a:
- Encontrar productos que se ajusten a sus necesidades
- Responder preguntas sobre especificaciones y características
- Hacer recomendaciones personalizadas
- Resolver dudas sobre envíos, garantías y devoluciones

Catálogo de productos disponibles:
1. Laptop HP Pavilion 15 - $899 - Intel i5, 8GB RAM, 256GB SSD, Pantalla 15.6"
2. Mouse Logitech MX Master 3 - $99 - Inalámbrico, Ergonómico, 7 botones programables
3. Teclado Mecánico Keychron K2 - $79 - Bluetooth, RGB, Switch Brown
4. Monitor Dell 27" 4K - $449 - IPS, HDR, 60Hz, USB-C
5. Webcam Logitech C920 - $79 - Full HD 1080p, Micrófono integrado
6. Auriculares Sony WH-1000XM5 - $399 - Cancelación de ruido, Bluetooth, 30h batería

Políticas de la tienda:
- Envío gratis en compras mayores a $100
- Devoluciones: 30 días desde la compra
- Garantía: 1 año en todos los productos
- Métodos de pago: Tarjetas de crédito/débito, PayPal

Responde de manera:
- Clara y concisa
- Amigable y profesional
- Basada en el catálogo proporcionado
- Si no tienes información, admítelo honestamente

Historial de conversación:
{chat_history}

Cliente: {input}
Asistente:"""

# Crear el prompt
prompt = PromptTemplate(
    input_variables=["chat_history", "input"],
    template=template
)

print("✅ Prompt template creado")

✅ Prompt template creado


## 🔗 Parte 7: Crear la Cadena de Conversación

Integramos LLM + Memoria + Prompt en una ConversationChain.

In [6]:
# Crear la cadena de conversación
conversation = ConversationChain(
    llm=llm,
    memory=memory,
    prompt=prompt,
    verbose=False  # Cambiar a True para ver el proceso interno
)

print("✅ Cadena de conversación creada")
print("   Componentes: LLM + Memoria + Prompt")

✅ Cadena de conversación creada
   Componentes: LLM + Memoria + Prompt


## 💬 Parte 8: Chatbot Interactivo - Clase Wrapper

Creamos una clase para manejar el chatbot de manera más elegante.

In [ ]:
class ChatbotEcommerce:
    """
    Chatbot para e-commerce con LangChain
    """
    
    def __init__(self, conversation_chain):
        self.conversation = conversation_chain
        self.num_interacciones = 0
    
    def chat(self, mensaje_usuario):
        """
        Envía un mensaje al chatbot y obtiene respuesta
        
        Args:
            mensaje_usuario (str): Mensaje del usuario
            
        Returns:
            str: Respuesta del chatbot
        """
        self.num_interacciones += 1
        
        try:
            respuesta = self.conversation.predict(input=mensaje_usuario)
            return respuesta
        except Exception as e:
            return f"Error: {str(e)}"
    
    def reset(self):
        """Reinicia la memoria del chatbot"""
        self.conversation.memory.clear()
        self.num_interacciones = 0
        print("🔄 Memoria reiniciada")
    
    def ver_historial(self):
        """Muestra el historial de conversación"""
        historial = self.conversation.memory.load_memory_variables({})
        print("\n📜 Historial de Conversación:")
        print("=" * 80)
        if 'chat_history' in historial:
            for msg in historial['chat_history']:
                msg_str = ""
                for ind in range(0, len(msg.content), 80):
                    msg_str += msg.content[ind:ind+80]+"\n"
                if isinstance(msg, HumanMessage):
                    print(f"👤 Usuario: {msg_str}")
                elif isinstance(msg, AIMessage):
                    print(f"🤖 Asistente: {msg_str}")
                    print("-" * 80)
        print(f"\nTotal de interacciones: {self.num_interacciones}")
        print("=" * 80)
    
    def estadisticas(self):
        """Muestra estadísticas de la conversación"""
        historial = self.conversation.memory.load_memory_variables({})
        num_mensajes = len(historial.get('chat_history', []))
        
        print("\n📊 Estadísticas:")
        print(f"   Total de mensajes en memoria: {num_mensajes}")
        print(f"   Interacciones del usuario: {self.num_interacciones}")

# Crear instancia del chatbot
chatbot = ChatbotEcommerce(conversation)

print("✅ Chatbot de E-commerce listo para usar")
print("\n💡 Comandos disponibles:")
print("   - chatbot.chat('tu mensaje'): Enviar mensaje")
print("   - chatbot.ver_historial(): Ver conversación completa")
print("   - chatbot.reset(): Reiniciar conversación")
print("   - chatbot.estadisticas(): Ver estadísticas")

✅ Chatbot de E-commerce listo para usar

💡 Comandos disponibles:
   - chatbot.chat('tu mensaje'): Enviar mensaje
   - chatbot.ver_historial(): Ver conversación completa
   - chatbot.reset(): Reiniciar conversación
   - chatbot.estadisticas(): Ver estadísticas


## 🧪 Parte 9: Probar el Chatbot

Simulemos una conversación real con el chatbot.

In [14]:
print("🤖 CHATBOT DE E-COMMERCE - TECHSTORE")
print("=" * 80)

# Conversación de ejemplo
mensajes = [
    "Hola, estoy buscando una laptop para trabajar desde casa",
    "¿Cuánto cuesta?",
    "¿Tiene envío gratis?",
    "Perfecto, también necesito un mouse inalámbrico. ¿Qué tienes?",
    "Me interesa el MX Master 3. ¿Cuál sería el total con ambos productos?"
]

for mensaje in mensajes:
    print(f"\n👤 USER: {mensaje}")
    respuesta = chatbot.chat(mensaje)
    print(f"🤖 ASSISTANT: {respuesta}")
    print("-" * 80)

🤖 CHATBOT DE E-COMMERCE - TECHSTORE

👤 USER: Hola, estoy buscando una laptop para trabajar desde casa
🤖 ASSISTANT: ¡Hola! ¡Bienvenido a TechStore! Claro, estaré encantado de ayudarte a encontrar la laptop ideal para trabajar desde casa. ¿Podrías decirme un poco más sobre tus necesidades? Por ejemplo:

- ¿Qué tipo de trabajo realizas (oficina, diseño gráfico, programación, etc.)?
- ¿Necesitas una batería duradera?
- ¿Tienes un presupuesto específico?

Esto me ayudará a recomendarte la mejor opción. 😊
--------------------------------------------------------------------------------

👤 USER: ¿Cuánto cuesta?
🤖 ASSISTANT: La laptop HP Pavilion 15 cuesta $899. Es una excelente opción para trabajar desde casa, ya que cuenta con un procesador Intel i5, 8GB de RAM y 256GB de SSD, lo que ofrece un buen equilibrio entre rendimiento y almacenamiento. ¿Te gustaría que te ayude a elegir entre otras opciones disponibles? 😊
-------------------------------------------------------------------------------

In [15]:
chatbot.ver_historial()


📜 Historial de Conversación:
👤 Usuario: Hola, estoy buscando una laptop para trabajar desde casa

--------------------------------------------------------------------------------
🤖 Asistente: ¡Hola! ¡Bienvenido a TechStore! Claro, estaré encantado de ayudarte a encontrar 
la laptop ideal para trabajar desde casa. ¿Podrías decirme un poco más sobre tus
 necesidades? Por ejemplo:

- ¿Qué tipo de trabajo realizas (oficina, diseño grá
fico, programación, etc.)?
- ¿Necesitas una batería duradera?
- ¿Tienes un presu
puesto específico?

Esto me ayudará a recomendarte la mejor opción. 😊

--------------------------------------------------------------------------------
👤 Usuario: ¿Cuánto cuesta?

--------------------------------------------------------------------------------
🤖 Asistente: La laptop HP Pavilion 15 cuesta $899. Es una excelente opción para trabajar desd
e casa, ya que cuenta con un procesador Intel i5, 8GB de RAM y 256GB de SSD, lo 
que ofrece un buen equilibrio entre rendimiento

## 📊 Parte 10: Ver Historial y Estadísticas

In [20]:
# Ver historial completo
chatbot.ver_historial()

# Ver estadísticas
chatbot.estadisticas()


📜 Historial de Conversación:
👤 Usuario: Hola, estoy buscando una laptop para trabajar desde casa
--------------------------------------------------------------------------------
🤖 Bot: ¡Hola! ¡Claro que sí! Para ayudarte mejor, ¿me podrías decir un poco más sobre lo que necesitas? Por ejemplo:

- ¿Qué tipo de trabajo realizas (oficina, diseño, programación, etc.)?
- ¿Tienes un presupuesto específico?
- ¿Prefieres una laptop ligera y portátil o algo más potente?

¡Estoy aquí para recomendarte lo mejor según tus necesidades! 😊
--------------------------------------------------------------------------------
👤 Usuario: ¿Cuánto cuesta?
--------------------------------------------------------------------------------
🤖 Bot: ¡Hola nuevamente! La laptop HP Pavilion 15 cuesta $899. Es una opción muy buena para trabajar desde casa, ya que cuenta con un procesador Intel i5, 8GB de RAM y 256GB de almacenamiento SSD, lo que la hace rápida y eficiente para tareas de oficina y uso general. ¿Te gustar

## 🔄 Parte 11: Comparación Completa de Tipos de Memoria

En LangChain existen **5 tipos principales de memoria**, cada uno diseñado para diferentes escenarios de uso. En esta sección compararemos todos los tipos disponibles:

1. **ConversationBufferMemory** - Almacena TODO el historial completo
2. **ConversationBufferWindowMemory** - Solo las últimas K interacciones
3. **ConversationSummaryMemory** - Resume la conversación progresivamente
4. **ConversationSummaryBufferMemory** - Híbrido: mensajes recientes + resumen de antiguos
5. **ConversationEntityMemory** - Rastrea entidades específicas (personas, lugares, productos)

Ejecuta la celda siguiente para ver ejemplos prácticos de cada tipo y una tabla comparativa.

In [ ]:
# ==============================================================================
# 🔄 TIPOS DE MEMORIA EN LANGCHAIN
# ==============================================================================
# LangChain ofrece diferentes tipos de memoria, cada uno con características únicas:
# 
# 1. ConversationBufferMemory - Almacena TODO el historial
# 2. ConversationBufferWindowMemory - Solo las últimas K interacciones
# 3. ConversationSummaryMemory - Resume la conversación completa
# 4. ConversationSummaryBufferMemory - Híbrido: mensajes recientes + resumen de antiguos
# 5. ConversationEntityMemory - Rastrea entidades específicas (personas, lugares, etc.)
# ==============================================================================

print("=" * 100)
print("🧠 COMPARACIÓN DE TIPOS DE MEMORIA EN LANGCHAIN")
print("=" * 100)

# ==============================================================================
# 1️⃣  CONVERSATIONBUFFERMEMORY - Memoria Completa
# ==============================================================================
print("\n" + "=" * 100)
print("1️⃣  CONVERSATIONBUFFERMEMORY - Almacena TODO el historial")
print("=" * 100)
print("✅ Ventajas: Máximo contexto, recuerda todo")
print("❌ Desventajas: Consume muchos tokens, puede ser costoso")
print("🎯 Uso ideal: Conversaciones cortas o cuando necesitas TODO el contexto\n")

memory_buffer = ConversationBufferMemory(
    return_messages=True,
    memory_key="chat_history"
)

llm_buffer = ChatOpenAI(
    model="qwen-turbo",
    temperature=0.7,
    max_tokens=500,
    base_url="https://dashscope-intl.aliyuncs.com/compatible-mode/v1"
)

conversation_buffer = ConversationChain(
    llm=llm_buffer,
    memory=memory_buffer,
    prompt=prompt,
    verbose=False
)

chatbot_buffer = ChatbotEcommerce(conversation_buffer)

# Test
mensajes_buffer = [
    "Hola, me llamo María y busco una laptop",
    "¿Cuál recomiendas?",
    "¿Recuerdas mi nombre?"
]

for msg in mensajes_buffer:
    print(f"👤 Usuario: {msg}")
    resp = chatbot_buffer.chat(msg)
    print(f"🤖 Bot: {resp}\n")
    print("-" * 100)

print("💡 Resultado: Recuerda TODO, incluyendo el nombre 'María'")

# ==============================================================================
# 2️⃣  CONVERSATIONBUFFERWINDOWMEMORY - Ventana Deslizante
# ==============================================================================
print("\n" + "=" * 100)
print("2️⃣  CONVERSATIONBUFFERWINDOWMEMORY - Solo últimas K interacciones")
print("=" * 100)
print("✅ Ventajas: Controla el uso de tokens, memoria eficiente")
print("❌ Desventajas: Olvida conversaciones antiguas")
print("🎯 Uso ideal: Conversaciones largas donde solo importa el contexto reciente\n")

memory_window = ConversationBufferWindowMemory(
    k=2,  # Solo las últimas 2 interacciones
    return_messages=True,
    memory_key="chat_history"
)

llm_window = ChatOpenAI(
    model="qwen-turbo",
    temperature=0.7,
    max_tokens=500,
    base_url="https://dashscope-intl.aliyuncs.com/compatible-mode/v1"
)

conversation_window = ConversationChain(
    llm=llm_window,
    memory=memory_window,
    prompt=prompt,
    verbose=False
)

chatbot_window = ChatbotEcommerce(conversation_window)

# Test con 4 mensajes (k=2 solo recordará los últimos 2)
mensajes_window = [
    "Hola, me llamo Juan",
    "Busco auriculares con cancelación de ruido",
    "¿Cuál es el precio?",
    "¿Recuerdas mi nombre?"  # Esta pregunta probará la memoria limitada
]

for msg in mensajes_window:
    print(f"👤 Usuario: {msg}")
    resp = chatbot_window.chat(msg)
    print(f"🤖 Bot: {resp}\n")
    print("-" * 100)

print("💡 Resultado: Con k=2, probablemente NO recuerda 'Juan' porque quedó fuera de la ventana")

# ==============================================================================
# 3️⃣  CONVERSATIONSUMMARYMEMORY - Resumen Progresivo
# ==============================================================================
print("\n" + "=" * 100)
print("3️⃣  CONVERSATIONSUMMARYMEMORY - Resume la conversación")
print("=" * 100)
print("✅ Ventajas: Mantiene contexto con pocos tokens, ideal para conversaciones largas")
print("❌ Desventajas: Pierde detalles específicos, requiere LLM para resumir")
print("🎯 Uso ideal: Conversaciones muy largas donde necesitas contexto general\n")

memory_summary = ConversationSummaryMemory(
    llm=ChatOpenAI(
        model="qwen-turbo",
        temperature=0.3,  # Más bajo para resúmenes consistentes
        max_tokens=300,
        base_url="https://dashscope-intl.aliyuncs.com/compatible-mode/v1"
    ),
    return_messages=True,
    memory_key="chat_history"
)

llm_summary = ChatOpenAI(
    model="qwen-turbo",
    temperature=0.7,
    max_tokens=500,
    base_url="https://dashscope-intl.aliyuncs.com/compatible-mode/v1"
)

conversation_summary = ConversationChain(
    llm=llm_summary,
    memory=memory_summary,
    prompt=prompt,
    verbose=False
)

chatbot_summary = ChatbotEcommerce(conversation_summary)

# Test
mensajes_summary = [
    "Hola, soy Pedro y necesito un monitor 4K",
    "Mi presupuesto es de máximo $500",
    "¿Qué opciones tengo?",
    "¿Recuerdas mi nombre y presupuesto?"
]

for msg in mensajes_summary:
    print(f"👤 Usuario: {msg}")
    resp = chatbot_summary.chat(msg)
    print(f"🤖 Bot: {resp}\n")
    print("-" * 100)

print("💡 Resultado: Resume la conversación, puede recordar información clave como nombre y presupuesto")

# ==============================================================================
# 4️⃣  CONVERSATIONSUMMARYBUFFERMEMORY - Híbrido
# ==============================================================================
print("\n" + "=" * 100)
print("4️⃣  CONVERSATIONSUMMARYBUFFERMEMORY - Mensajes recientes + Resumen de antiguos")
print("=" * 100)
print("✅ Ventajas: Balance perfecto entre detalle y eficiencia")
print("❌ Desventajas: Más complejo de configurar")
print("🎯 Uso ideal: Conversaciones largas donde necesitas detalles recientes + contexto histórico\n")

memory_summary_buffer = ConversationSummaryBufferMemory(
    llm=ChatOpenAI(
        model="qwen-turbo",
        temperature=0.3,
        max_tokens=300,
        base_url="https://dashscope-intl.aliyuncs.com/compatible-mode/v1"
    ),
    max_token_limit=200,  # Cuando excede este límite, resume los mensajes antiguos
    return_messages=True,
    memory_key="chat_history"
)

llm_summary_buffer = ChatOpenAI(
    model="qwen-turbo",
    temperature=0.7,
    max_tokens=500,
    base_url="https://dashscope-intl.aliyuncs.com/compatible-mode/v1"
)

conversation_summary_buffer = ConversationChain(
    llm=llm_summary_buffer,
    memory=memory_summary_buffer,
    prompt=prompt,
    verbose=False
)

chatbot_summary_buffer = ChatbotEcommerce(conversation_summary_buffer)

# Test
mensajes_summary_buffer = [
    "Hola, me llamo Ana y trabajo en diseño gráfico",
    "Necesito una laptop potente para renderizado",
    "También un monitor con buena precisión de color",
    "¿Qué me recomiendas?",
    "¿Recuerdas mi nombre y profesión?"
]

for msg in mensajes_summary_buffer:
    print(f"👤 Usuario: {msg}")
    resp = chatbot_summary_buffer.chat(msg)
    print(f"🤖 Bot: {resp}\n")
    print("-" * 100)

print("💡 Resultado: Mantiene mensajes recientes completos y resume los antiguos")

# ==============================================================================
# 5️⃣  CONVERSATIONENTITYMEMORY - Seguimiento de Entidades
# ==============================================================================
print("\n" + "=" * 100)
print("5️⃣  CONVERSATIONENTITYMEMORY - Rastrea entidades específicas")
print("=" * 100)
print("✅ Ventajas: Excelente para recordar detalles sobre personas, lugares, productos")
print("❌ Desventajas: Requiere LLM adicional para extraer entidades")
print("🎯 Uso ideal: Soporte al cliente, CRM, asistentes personalizados\n")

memory_entity = ConversationEntityMemory(
    llm=ChatOpenAI(
        model="qwen-turbo",
        temperature=0.3,
        max_tokens=300,
        base_url="https://dashscope-intl.aliyuncs.com/compatible-mode/v1"
    ),
    return_messages=True,
    memory_key="chat_history"
)

llm_entity = ChatOpenAI(
    model="qwen-turbo",
    temperature=0.7,
    max_tokens=500,
    base_url="https://dashscope-intl.aliyuncs.com/compatible-mode/v1"
)

conversation_entity = ConversationChain(
    llm=llm_entity,
    memory=memory_entity,
    prompt=prompt,
    verbose=False
)

chatbot_entity = ChatbotEcommerce(conversation_entity)

# Test
mensajes_entity = [
    "Hola, soy Carlos de la empresa TechCorp",
    "Necesito comprar 5 laptops HP Pavilion 15",
    "También 10 mouse Logitech",
    "¿Cuál sería el total y hay descuento por volumen?",
    "¿Recuerdas mi nombre, empresa y qué productos quiero?"
]

for msg in mensajes_entity:
    print(f"👤 Usuario: {msg}")
    resp = chatbot_entity.chat(msg)
    print(f"🤖 Bot: {resp}\n")
    print("-" * 100)

print("💡 Resultado: Rastrea entidades como nombre (Carlos), empresa (TechCorp), productos, cantidades")

# ==============================================================================
# 📊 TABLA COMPARATIVA FINAL
# ==============================================================================
print("\n" + "=" * 100)
print("📊 TABLA COMPARATIVA DE TIPOS DE MEMORIA")
print("=" * 100)
print("""
┌──────────────────────────────┬────────────────┬──────────────────┬───────────────────────────┐
│ Tipo de Memoria              │ Uso de Tokens  │ Retención        │ Mejor Caso de Uso         │
├──────────────────────────────┼────────────────┼──────────────────┼───────────────────────────┤
│ ConversationBufferMemory     │ Alto           │ 100%             │ Chats cortos              │
│ ConversationBufferWindow     │ Medio          │ Últimas K msgs   │ Chats largos, contexto    │
│                              │                │                  │ reciente importante       │
│ ConversationSummaryMemory    │ Bajo           │ Resumen general  │ Chats muy largos          │
│ ConversationSummaryBuffer    │ Medio-Bajo     │ Híbrido          │ Balance contexto/costo    │
│ ConversationEntityMemory     │ Medio          │ Entidades clave  │ CRM, soporte, ventas      │
└──────────────────────────────┴────────────────┴──────────────────┴───────────────────────────┘
""")

In [ ]:
import json
from typing import Dict, List, Any
from datetime import datetime
import os

# ==============================================================================
# 📋 EXTRACCIÓN DE DATOS ESTRUCTURADOS EN JSON
# ==============================================================================

print("=" * 100)
print("📋 CHATBOT CON RESPUESTAS ESTRUCTURADAS EN JSON")
print("=" * 100)

# ==============================================================================
# PASO 1: Crear un catálogo de productos estructurado
# ==============================================================================
CATALOGO_PRODUCTOS = {
    "PROD-001": {
        "nombre": "Laptop HP Pavilion 15",
        "precio": 899,
        "categoria": "laptops",
        "especificaciones": "Intel i5, 8GB RAM, 256GB SSD, Pantalla 15.6\""
    },
    "PROD-002": {
        "nombre": "Mouse Logitech MX Master 3",
        "precio": 99,
        "categoria": "accesorios",
        "especificaciones": "Inalámbrico, Ergonómico, 7 botones programables"
    },
    "PROD-003": {
        "nombre": "Teclado Mecánico Keychron K2",
        "precio": 79,
        "categoria": "accesorios",
        "especificaciones": "Bluetooth, RGB, Switch Brown"
    },
    "PROD-004": {
        "nombre": "Monitor Dell 27\" 4K",
        "precio": 449,
        "categoria": "monitores",
        "especificaciones": "IPS, HDR, 60Hz, USB-C"
    },
    "PROD-005": {
        "nombre": "Webcam Logitech C920",
        "precio": 79,
        "categoria": "accesorios",
        "especificaciones": "Full HD 1080p, Micrófono integrado"
    },
    "PROD-006": {
        "nombre": "Auriculares Sony WH-1000XM5",
        "precio": 399,
        "categoria": "audio",
        "especificaciones": "Cancelación de ruido, Bluetooth, 30h batería"
    }
}

# Crear información del catálogo para el prompt
catalogo_info = "\n".join([
    f"{codigo}. {info['nombre']} - ${info['precio']} - {info['especificaciones']}"
    for codigo, info in CATALOGO_PRODUCTOS.items()
])

# ==============================================================================
# PASO 2: Crear un prompt que solicite respuesta en JSON
# ==============================================================================
json_extraction_template = """Eres un asistente de ventas para TechStore que extrae información de conversaciones con clientes.

Tu tarea es DOBLE:
1. Responder de manera amigable y profesional al cliente (en el campo "respuesta_cliente")
2. Extraer información estructurada de la conversación (en los demás campos JSON)

Catálogo de productos:
{catalogo_info}

IMPORTANTE: Debes responder ÚNICAMENTE con un objeto JSON válido con la siguiente estructura:
{{
    "respuesta_cliente": "Tu respuesta amigable al cliente aquí",
    "cliente": {{
        "nombre": "nombre del cliente si lo mencionó, o null",
        "empresa": "empresa si la mencionó, o null"
    }},
    "productos_mencionados": [
        {{
            "codigo": "PROD-XXX",
            "nombre": "Nombre del producto",
            "cantidad": 1,
            "precio_unitario": 0
        }}
    ],
    "intencion": "consulta|cotizacion|compra",
    "requiere_envio": true|false,
    "total_estimado": 0,
    "notas_adicionales": "cualquier información relevante"
}}

Historial de conversación:
{chat_history}

Cliente: {input}

JSON de respuesta:"""

# Crear el prompt con partial_variables para incluir el catálogo de forma fija
# Esto permite que ConversationChain solo vea las variables chat_history e input
json_prompt = PromptTemplate(
    input_variables=["chat_history", "input"],
    template=json_extraction_template,
    partial_variables={"catalogo_info": catalogo_info}
)

print("✅ Prompt JSON creado con catálogo incluido")

# ==============================================================================
# PASO 3: Crear LLM configurado para JSON
# ==============================================================================
llm_json = ChatOpenAI(
    model="qwen-turbo",
    temperature=0.3,  # Más bajo para respuestas más consistentes
    max_tokens=800,   # Más tokens para respuestas JSON detalladas
    base_url="https://dashscope-intl.aliyuncs.com/compatible-mode/v1"
)

# Crear memoria para el contexto
memory_json = ConversationBufferMemory(
    return_messages=True,
    memory_key="chat_history"
)

# Crear la cadena de conversación
conversation_json = ConversationChain(
    llm=llm_json,
    memory=memory_json,
    prompt=json_prompt,
    verbose=False
)

print("✅ ConversationChain para JSON creada correctamente")

# ==============================================================================
# PASO 4: Función para procesar y parsear respuestas JSON
# ==============================================================================
def extraer_json_de_respuesta(respuesta: str) -> Dict[str, Any]:
    """
    Extrae y parsea JSON de la respuesta del LLM
    """
    try:
        # Intentar parsear directamente
        return json.loads(respuesta)
    except json.JSONDecodeError:
        # Si falla, intentar extraer JSON entre llaves
        import re
        json_match = re.search(r'\{.*\}', respuesta, re.DOTALL)
        if json_match:
            try:
                return json.loads(json_match.group())
            except:
                pass
        
        # Si todo falla, retornar estructura vacía
        return {
            "respuesta_cliente": respuesta,
            "error": "No se pudo parsear JSON"
        }

def procesar_conversacion_json(mensaje: str) -> tuple:
    """
    Procesa un mensaje y retorna la respuesta + datos estructurados
    """
    # Obtener respuesta del LLM
    respuesta_raw = conversation_json.predict(input=mensaje)
    
    # Parsear JSON
    datos_estructurados = extraer_json_de_respuesta(respuesta_raw)
    
    # Obtener respuesta para el cliente
    respuesta_cliente = datos_estructurados.get("respuesta_cliente", respuesta_raw)
    
    return respuesta_cliente, datos_estructurados

# ==============================================================================
# PASO 5: Probar con conversaciones de ejemplo
# ==============================================================================
print("\n" + "=" * 100)
print("🧪 EJEMPLOS DE EXTRACCIÓN DE DATOS ESTRUCTURADOS")
print("=" * 100)

conversaciones_ejemplo = [
    "Hola, soy María González de la empresa TechCorp y necesito cotizar 3 laptops HP Pavilion",
    "También necesitamos 5 mouse Logitech para la oficina",
    "¿Cuál sería el total con envío incluido?",
    "Perfecto, quiero proceder con la compra"
]

# Procesar cada mensaje
datos_extraidos_totales = []

for i, mensaje in enumerate(conversaciones_ejemplo, 1):
    print(f"\n{'─' * 100}")
    print(f"💬 Mensaje {i}: {mensaje}")
    print(f"{'─' * 100}")
    
    # Procesar mensaje
    respuesta, datos = procesar_conversacion_json(mensaje)
    
    # Mostrar respuesta al cliente
    print(f"\n🤖 Respuesta al Cliente:")
    print(f"   {respuesta}")
    
    # Mostrar datos estructurados extraídos
    print(f"\n📊 Datos Estructurados Extraídos:")
    print(json.dumps(datos, indent=4, ensure_ascii=False))
    
    datos_extraidos_totales.append(datos)

# ==============================================================================
# PASO 6: Resumen de información extraída
# ==============================================================================
print("\n" + "=" * 100)
print("📈 RESUMEN DE INFORMACIÓN EXTRAÍDA DE TODA LA CONVERSACIÓN")
print("=" * 100)

# Extraer información consolidada
cliente_nombre = None
cliente_empresa = None
productos_totales = []
intencion_final = None

for datos in datos_extraidos_totales:
    # Extraer información del cliente
    if datos.get("cliente"):
        if datos["cliente"].get("nombre"):
            cliente_nombre = datos["cliente"]["nombre"]
        if datos["cliente"].get("empresa"):
            cliente_empresa = datos["cliente"]["empresa"]
    
    # Acumular productos
    if datos.get("productos_mencionados"):
        productos_totales.extend(datos["productos_mencionados"])
    
    # Última intención
    if datos.get("intencion"):
        intencion_final = datos["intencion"]

# Calcular total
total_pedido = sum(
    p.get("precio_unitario", 0) * p.get("cantidad", 1) 
    for p in productos_totales
)

# Mostrar resumen
resumen = {
    "cliente": {
        "nombre": cliente_nombre,
        "empresa": cliente_empresa
    },
    "productos_solicitados": productos_totales,
    "cantidad_items": len(productos_totales),
    "total_estimado": total_pedido,
    "intencion_final": intencion_final,
    "requiere_seguimiento": intencion_final == "compra"
}

print(json.dumps(resumen, indent=4, ensure_ascii=False))

# ==============================================================================
# PASO 7: Funciones para guardar en JSON (IMPLEMENTACIÓN REAL)
# ==============================================================================
print("\n" + "=" * 100)
print("🔗 EJEMPLO: INTEGRACIÓN CON SISTEMA BACKEND (GUARDADO EN JSON)")
print("=" * 100)

# Nombre del archivo JSON donde se guardarán los pedidos
ARCHIVO_PEDIDOS = "pedidos_techstore.json"

def cargar_pedidos_existentes() -> List[Dict[str, Any]]:
    """
    Carga los pedidos existentes desde el archivo JSON
    """
    if os.path.exists(ARCHIVO_PEDIDOS):
        try:
            with open(ARCHIVO_PEDIDOS, 'r', encoding='utf-8') as f:
                return json.load(f)
        except:
            return []
    return []

def generar_id_pedido() -> str:
    """
    Genera un ID único para el pedido basado en timestamp
    """
    timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
    return f"PED-{timestamp}"

def guardar_pedido_en_bd(datos_pedido: Dict[str, Any]) -> Dict[str, Any]:
    """
    Guarda el pedido en un archivo JSON real
    
    Args:
        datos_pedido: Diccionario con información del pedido
        
    Returns:
        Diccionario con pedido_id y status
    """
    print("\n📝 Guardando pedido en archivo JSON...")
    
    # Cargar pedidos existentes
    pedidos = cargar_pedidos_existentes()
    
    # Generar ID único
    pedido_id = generar_id_pedido()
    
    # Crear estructura completa del pedido
    pedido_completo = {
        "pedido_id": pedido_id,
        "fecha_creacion": datetime.now().isoformat(),
        "status": "pending",
        "cliente": datos_pedido.get("cliente", {}),
        "productos": datos_pedido.get("productos_solicitados", []),
        "cantidad_items": datos_pedido.get("cantidad_items", 0),
        "total_estimado": datos_pedido.get("total_estimado", 0),
        "intencion": datos_pedido.get("intencion_final", ""),
        "requiere_seguimiento": datos_pedido.get("requiere_seguimiento", False)
    }
    
    # Agregar nuevo pedido a la lista
    pedidos.append(pedido_completo)
    
    # Guardar en archivo JSON
    with open(ARCHIVO_PEDIDOS, 'w', encoding='utf-8') as f:
        json.dump(pedidos, f, indent=4, ensure_ascii=False)
    
    # Mostrar información
    print(f"   ✅ Pedido guardado correctamente")
    print(f"   📄 Archivo: {ARCHIVO_PEDIDOS}")
    print(f"   🆔 ID Pedido: {pedido_id}")
    print(f"   👤 Cliente: {datos_pedido['cliente']['nombre']}")
    print(f"   🏢 Empresa: {datos_pedido['cliente']['empresa']}")
    print(f"   📦 Total items: {datos_pedido['cantidad_items']}")
    print(f"   💰 Total: ${datos_pedido['total_estimado']}")
    print(f"   📊 Estado: pending")
    
    return {"pedido_id": pedido_id, "status": "pending"}

def enviar_notificacion_ventas(datos_pedido: Dict[str, Any]):
    """
    Simula enviar notificación al equipo de ventas
    También guarda un log en archivo
    """
    print("\n📧 Enviando notificación al equipo de ventas...")
    
    notificacion = {
        "timestamp": datetime.now().isoformat(),
        "tipo": "nueva_oportunidad",
        "asunto": f"Nueva oportunidad de venta - {datos_pedido['cliente']['empresa']}",
        "cliente": datos_pedido['cliente']['nombre'],
        "empresa": datos_pedido['cliente']['empresa'],
        "monto_estimado": datos_pedido['total_estimado'],
        "productos": datos_pedido['productos_solicitados']
    }
    
    # Guardar notificación en archivo de log
    archivo_log = "notificaciones_ventas.json"
    
    # Cargar notificaciones existentes
    notificaciones = []
    if os.path.exists(archivo_log):
        try:
            with open(archivo_log, 'r', encoding='utf-8') as f:
                notificaciones = json.load(f)
        except:
            notificaciones = []
    
    # Agregar nueva notificación
    notificaciones.append(notificacion)
    
    # Guardar en archivo
    with open(archivo_log, 'w', encoding='utf-8') as f:
        json.dump(notificaciones, f, indent=4, ensure_ascii=False)
    
    print(f"   ✅ Notificación enviada y guardada")
    print(f"   �� Archivo: {archivo_log}")
    print(f"   📬 Asunto: {notificacion['asunto']}")
    print(f"   💵 Monto estimado: ${datos_pedido['total_estimado']}")

def mostrar_estadisticas_pedidos():
    """
    Muestra estadísticas de los pedidos guardados
    """
    if not os.path.exists(ARCHIVO_PEDIDOS):
        print("\n📊 No hay pedidos guardados aún")
        return
    
    pedidos = cargar_pedidos_existentes()
    
    if not pedidos:
        print("\n📊 No hay pedidos guardados aún")
        return
    
    print("\n" + "=" * 100)
    print("📊 ESTADÍSTICAS DE PEDIDOS GUARDADOS")
    print("=" * 100)
    
    total_pedidos = len(pedidos)
    total_ingresos = sum(p.get("total_estimado", 0) for p in pedidos)
    total_items = sum(p.get("cantidad_items", 0) for p in pedidos)
    
    print(f"\n📈 Resumen General:")
    print(f"   Total de pedidos: {total_pedidos}")
    print(f"   Total items vendidos: {total_items}")
    print(f"   Ingresos totales: ${total_ingresos:,.2f}")
    print(f"   Promedio por pedido: ${total_ingresos/total_pedidos:,.2f}")
    
    print(f"\n📋 Últimos 3 pedidos:")
    for pedido in pedidos[-3:]:
        print(f"\n   🆔 {pedido['pedido_id']}")
        print(f"      Cliente: {pedido['cliente'].get('nombre', 'N/A')}")
        print(f"      Empresa: {pedido['cliente'].get('empresa', 'N/A')}")
        print(f"      Total: ${pedido.get('total_estimado', 0)}")
        print(f"      Fecha: {pedido.get('fecha_creacion', 'N/A')[:19]}")

# ==============================================================================
# PASO 8: Guardar el pedido y enviar notificación
# ==============================================================================

# Simular integración
if resumen["intencion_final"] == "compra":
    resultado_bd = guardar_pedido_en_bd(resumen)
    enviar_notificacion_ventas(resumen)
    print(f"\n🎉 Pedido procesado exitosamente: {resultado_bd['pedido_id']}")
    
    # Mostrar estadísticas
    mostrar_estadisticas_pedidos()
else:
    print("\n💡 La intención no es 'compra', no se guardará el pedido")
    print(f"   Intención actual: {resumen['intencion_final']}")

# ==============================================================================
# 💡 VENTAJAS DE USAR JSON ESTRUCTURADO
# ==============================================================================
print("\n" + "=" * 100)
print("💡 VENTAJAS DE USAR RESPUESTAS JSON ESTRUCTURADAS")
print("=" * 100)
print("""
✅ Integración fácil con sistemas backend (CRM, ERP, bases de datos)
✅ Análisis y reportes automatizados
✅ Validación de datos más robusta
✅ Procesamiento programático de entidades
✅ Seguimiento estructurado de conversaciones
✅ Automatización de workflows (pedidos, cotizaciones, seguimientos)
✅ Analytics y métricas de ventas en tiempo real
✅ Facilita el testing y debugging

🎯 Casos de uso:
- Sistemas de pedidos automatizados
- CRM integrado con chatbot
- Analytics de conversaciones
- Automatización de cotizaciones
- Gestión de inventario en tiempo real
- Reportes de ventas y tendencias

📁 Archivos generados en este ejemplo:
- pedidos_techstore.json: Base de datos de pedidos
- notificaciones_ventas.json: Log de notificaciones enviadas
""")

## 📋 Parte 11.5: Extracción de Datos Estructurados en JSON

En aplicaciones reales, a menudo necesitamos extraer información específica de la conversación en un formato estructurado (JSON) para procesarla programáticamente. Por ejemplo:

- **Códigos de productos** mencionados
- **Nombres de clientes**
- **Cantidades** solicitadas
- **Precios** y totales
- **Intención de compra** (consulta, cotización, compra)
- **Entidades extraídas** (fechas, ubicaciones, etc.)

Esta sección muestra cómo configurar el LLM para que devuelva respuestas en formato JSON que pueden ser parseadas y utilizadas por otros sistemas (bases de datos, CRM, sistemas de pedidos, etc.).

## 🎭 Parte 12: Chatbot Interactivo en Consola

Crea un loop interactivo para chatear en tiempo real.

In [ ]:
def iniciar_chat_interactivo():
    """
    Inicia un chat interactivo en la consola
    """
    # Crear nuevo LLM para la sesión interactiva
    llm_interactivo = ChatOpenAI(
        model="qwen-turbo",
        temperature=0.7,
        max_tokens=500,
        base_url="https://dashscope-intl.aliyuncs.com/compatible-mode/v1"
    )
    
    # Crear nueva memoria para sesión interactiva
    memory_interactivo = ConversationBufferMemory(
        return_messages=True,
        memory_key="chat_history"
    )
    
    conversation_interactivo = ConversationChain(
        llm=llm_interactivo,
        memory=memory_interactivo,
        prompt=prompt,
        verbose=False
    )
    
    chatbot_interactivo = ChatbotEcommerce(conversation_interactivo)
    
    print("\n" + "=" * 80)
    print("🤖 CHATBOT INTERACTIVO - TECHSTORE (Powered by Alibaba Qwen)")
    print("=" * 80)
    print("¡Bienvenido a TechStore! Soy tu asistente virtual.")
    print("Escribe 'salir' para terminar la conversación.")
    print("Escribe 'historial' para ver la conversación completa.")
    print("=" * 80 + "\n")
    
    while True:
        try:
            mensaje = input("👤 Tú: ").strip()
            
            if not mensaje:
                continue
            
            if mensaje.lower() in ['salir', 'exit', 'quit']:
                print("\n🤖 Bot: ¡Gracias por visitar TechStore! Que tengas un excelente día.")
                break
            
            if mensaje.lower() == 'historial':
                chatbot_interactivo.ver_historial()
                continue
            
            # Generar respuesta
            respuesta = chatbot_interactivo.chat(mensaje)
            print(f"\n🤖 Bot: {respuesta}\n")
            print("-" * 80)
            
        except KeyboardInterrupt:
            print("\n\n🤖 Bot: ¡Hasta pronto!")
            break
        except Exception as e:
            print(f"\n❌ Error: {e}")
            print("Por favor, intenta de nuevo.\n")
    
    # Mostrar estadísticas finales
    chatbot_interactivo.estadisticas()

# Para iniciar el chat interactivo, ejecuta:
# iniciar_chat_interactivo()

print("✅ Función de chat interactivo lista (usando Alibaba DashScope)")
print("\n💡 Para iniciar el chat interactivo, ejecuta: iniciar_chat_interactivo()")

## 🎓 Parte 13: Ejercicios Propuestos

### Ejercicio 1: Expandir el Catálogo
Agrega más productos al catálogo en el prompt template.

### Ejercicio 2: Sistema de Descuentos
Implementa un sistema que aplique descuentos automáticos basados en:
- Monto total de compra
- Cantidad de productos
- Productos en combo

### Ejercicio 3: Filtrado Inteligente
Crea un sistema que filtre productos por:
- Rango de precios
- Categoría
- Características específicas

### Ejercicio 4: Integración con Base de Datos
Conecta el chatbot a una base de datos real (SQLite) para obtener información de productos dinámicamente.

### Ejercicio 5: Sentiment Analysis
Agrega análisis de sentimiento para detectar clientes insatisfechos y escalar a soporte humano.

---

## 🆓 ALTERNATIVA: Usando Modelos Locales (Sin API Key)

Si no tienes API key de OpenAI, puedes usar modelos locales de HuggingFace.

In [ ]:
# Alternativa con HuggingFace (sin costo)
# !pip install langchain-huggingface -q

# from langchain_huggingface import HuggingFacePipeline
# from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# # Cargar modelo local
# model_id = "google/flan-t5-base"
# tokenizer = AutoTokenizer.from_pretrained(model_id)
# model = AutoModelForCausalLM.from_pretrained(model_id)

# # Crear pipeline
# pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer, max_length=200)

# # Crear LLM de LangChain
# llm_local = HuggingFacePipeline(pipeline=pipe)

# # Usar igual que antes
# conversation_local = ConversationChain(
#     llm=llm_local,
#     memory=ConversationBufferMemory(),
#     prompt=prompt
# )

print("💡 Código de alternativa con HuggingFace disponible (comentado)")

## 💡 Conclusiones

En este taller has aprendido a:

✅ Configurar LangChain con OpenAI GPT  
✅ Implementar gestión de memoria conversacional  
✅ Diseñar prompts efectivos para chatbots especializados  
✅ Crear cadenas de procesamiento (ConversationChain)  
✅ Construir un chatbot funcional para e-commerce  
✅ Comparar diferentes estrategias de memoria

### 🚀 Próximos Pasos

- Integrar con bases de datos reales
- Implementar RAG para consultar documentos
- Agregar herramientas externas (APIs, calculadoras)
- Desplegar en producción con Telegram/WhatsApp
- Implementar agentes autónomos

### 📚 Recursos Adicionales

- [LangChain Documentation](https://python.langchain.com/)
- [OpenAI API Reference](https://platform.openai.com/docs/)
- [LangChain Cookbook](https://github.com/langchain-ai/langchain/tree/master/cookbook)

---

**¡Felicitaciones!** 🎉 Has completado el Taller de la Sesión 4.